<a href="https://colab.research.google.com/github/h2o-polo5/invest-repo/blob/main/Santa_gk1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

santa_2024_path = kagglehub.competition_download('santa-2024')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    'Minimizing perplexity, a task quite grand'
    'A neural network’s mission, across the land'
    'A model, trained on texts, both vast and deep'
    'To rearrange the words, while others sleep'
    'It sorts and sifts, with algorithms bright'
    'To find the clearest path, the words made right'
    'A digital reindeer, with circuits aglow'
    'Reducing confusion, a wondrous show'
    'No longer puzzled, the reader’s mind at ease'
    'As the model’s magic, effortlessly, please'
    'A gift of clarity, a present divine'
    'A neural network’s work, a clever design'
]

tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    'Minimizing perplexity, a task quite grand',
    'A neural network’s mission, across the land.',
    'A model, trained on texts, both vast and deep',
    'To rearrange the words, while others sleep.'
    'It sorts and sifts, with algorithms bright',
    'To find the clearest path, the words made right.'
    'A digital reindeer, with circuits aglow',
    'Reducing confusion, a wondrous show.'
    'No longer puzzled, the reader’s mind at ease',
    'As the model’s magic, effortlessly, please.'
    'A gift of clarity, a present divine',
    'A neural network’s work, a clever design.'
]

tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

print(word_index)
print(sequences)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
    'Minimizing perplexity, a task quite grand'
    'A neural network’s mission, across the land'
    'A model, trained on texts, both vast and deep'
    'To rearrange the words, while others sleep'
    'It sorts and sifts, with algorithms bright'
    'To find the clearest path, the words made right'
    'A digital reindeer, with circuits aglow'
    'Reducing confusion, a wondrous show'
    'No longer puzzled, the reader’s mind at ease'
    'As the model’s magic, effortlessly, please'
    'A gift of clarity, a present divine'
    'A neural network’s work, a clever design'
]

tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

padded = pad_sequences(sequences)
print(word_index)
print(sequences)
print(padded)

In [ ]:
#embeddings

import numpy as np
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

def predict_order(sentences):
    n = len(sentences)
    scores = np.zeros((n, n))

    # Calculate scores for all pairs of sentences
    for i in range(n):
        for j in range(n):
            if i != j:
                # Encode sentences
                inputs = tokenizer(sentences[i], sentences[j], return_tensors='pt')
                outputs = model(**inputs)
                logits = outputs.logits
                scores[i][j] = logits[0][1].item()  # Get score for 'is next'

    # Order sentences based on scores
    order = list(range(n))
    order.sort(key=lambda x: sum(scores[x]), reverse=True)

    return [sentences[i] for i in order]

# Example scrambled sentences
scrambled_sentences = [
    "Minimizing perplexity, a task quite grand,",
    "A neural network’s mission, across the land.",
    "A model, trained on texts, both vast and deep,",
    "To rearrange the words, while others sleep.",
    "It sorts and sifts, with algorithms bright,",
    "To find the clearest path, the words made right.",
    "A digital reindeer, with circuits aglow,",
    "Reducing confusion, a wondrous show.",
    "No longer puzzled, the reader’s mind at ease,",
    "As the model’s magic, effortlessly, please.",
    "A gift of clarity, a present divine,",
    "A neural network’s work, a clever design."
]

# Predict and print ordered sentences
ordered_sentences = predict_order(scrambled_sentences)
for sentence in ordered_sentences:
    print(sentence)


In [ ]:
import gc
import os
from math import exp
from collections import Counter
from typing import List, Union
import numpy as np
import pandas as pd
import transformers
import torch

# Set environment variables for optimal performance
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ParticipantVisibleError(Exception):
    pass

class PerplexityCalculator:
    """Calculates perplexity of text using a pre-trained language model."""

    def __init__(self, model_path: str, load_in_8bit: bool = False):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

        # Load model with or without 8-bit quantization
        if load_in_8bit:
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path, quantization_config=quantization_config).to(DEVICE)
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(model_path).to(DEVICE)

        self.model.eval()

    def get_perplexity(self, input_texts: Union[str, List[str]], batch_size: int = 4) -> Union[float, List[float]]:
        """Calculates the perplexity of given texts."""
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []
        with torch.no_grad():
            for i in range(0, len(input_texts), batch_size):
                batch_texts = input_texts[i:i + batch_size]
                for text in batch_texts:
                    # Add special tokens for boundary conditions
                    text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
                    model_inputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False)

                    # Move inputs to the appropriate device
                    model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                    # Get model output and calculate loss
                    output = self.model(**model_inputs)
                    logits = output.logits

                    # Shift logits and labels for calculating loss
                    shift_logits = logits[..., :-1, :].contiguous()
                    shift_labels = model_inputs['input_ids'][..., 1:].contiguous()

                    loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
                    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

                    # Calculate average loss
                    sequence_loss = loss.sum() / len(loss)
                    loss_list.append(sequence_loss.cpu().item())

        # Calculate perplexities from losses
        perplexities = [exp(i) for i in loss_list]
        return perplexities[0] if single_input else perplexities

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        del self.model
        gc.collect()
        torch.cuda.empty_cache()

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str,
          model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
          load_in_8bit: bool = False, clear_mem: bool = False) -> float:
    """Calculates the mean perplexity of submitted text permutations compared to an original text."""

    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)
    invalid_mask = sol_counts != sub_counts

    if invalid_mask.any():
        raise ParticipantVisibleError('At least one submitted string is not a valid permutation of the solution string.')

    # Calculate perplexity for the submitted strings
    sub_strings = [' '.join(s.split()) for s in submission['text'].tolist()]  # Normalize whitespace

    scorer = PerplexityCalculator(model_path=model_path, load_in_8bit=load_in_8bit)

    # Get perplexities for each submitted string
    perplexities = scorer.get_perplexity(sub_strings)

    if clear_mem:
        try:
            scorer.clear_gpu_memory()
        except Exception as e:
            print(f'GPU memory clearing failed: {e}')

    return float(np.mean(perplexities))

# Example usage with provided sentences (replace with actual DataFrame inputs)
model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
solution_df = pd.DataFrame({
    'id': [0],
    'text': ["Minimizing perplexity, a task quite grand. A neural network’s mission, across the land. A model, trained on texts, both vast and deep. To rearrange the words, while others sleep. It sorts and sifts, with algorithms bright. To find the clearest path, the words made right. A digital reindeer, with circuits aglow. Reducing confusion, a wondrous show. No longer puzzled, the reader’s mind at ease. As the model’s magic, effortlessly please. A gift of clarity, a present divine. A neural network’s work, a clever design."]
})

submission_df = pd.DataFrame({
    'id': [0],
    'text': ["A neural network’s mission, across the land. Minimizing perplexity, a task quite grand. It sorts and sifts, with algorithms bright. To find the clearest path, the words made right."]
})

# Calculate score (perplexity) for submission against solution
perplexity_score = score(solution_df, submission_df, 'id', clear_mem=True)
print(f'Mean Perplexity Score: {perplexity_score}')


In [ ]:
import gc
import os
from math import exp
from collections import Counter
from typing import List, Union
import numpy as np
import pandas as pd
import transformers
import torch

# Set environment variables for optimal performance
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ParticipantVisibleError(Exception):
    pass

class PerplexityCalculator:
    """Calculates perplexity of text using a pre-trained language model."""

    def __init__(self, model_path: str, load_in_8bit: bool = False):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

        # Load model with or without 8-bit quantization
        if load_in_8bit:
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path, quantization_config=quantization_config).to(DEVICE)
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(model_path).to(DEVICE)

        self.model.eval()

    def get_perplexity(self, input_texts: Union[str, List[str]], batch_size: int = 4) -> Union[float, List[float]]:
        """Calculates the perplexity of given texts."""
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []
        with torch.no_grad():
            for i in range(0, len(input_texts), batch_size):
                batch_texts = input_texts[i:i + batch_size]
                for text in batch_texts:
                    # Add special tokens for boundary conditions
                    text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
                    model_inputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False)

                    # Move inputs to the appropriate device
                    model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                    # Get model output and calculate loss
                    output = self.model(**model_inputs)
                    logits = output.logits

                    # Shift logits and labels for calculating loss
                    shift_logits = logits[..., :-1, :].contiguous()
                    shift_labels = model_inputs['input_ids'][..., 1:].contiguous()

                    loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
                    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

                    # Calculate average loss
                    sequence_loss = loss.sum() / len(loss)
                    loss_list.append(sequence_loss.cpu().item())

        # Calculate perplexities from losses
        perplexities = [exp(i) for i in loss_list]
        return perplexities[0] if single_input else perplexities

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        del self.model
        gc.collect()
        torch.cuda.empty_cache()

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str,
          model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
          load_in_8bit: bool = False, clear_mem: bool = False) -> float:
    """Calculates the mean perplexity of submitted text permutations compared to an original text."""

    # Normalize text by lowercasing and stripping whitespace
    solution['text'] = solution['text'].str.lower().str.strip()
    submission['text'] = submission['text'].str.lower().str.strip()

    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)

    invalid_mask = sol_counts != sub_counts

    if invalid_mask.any():
        print("Mismatch found:")
        for idx in invalid_mask[invalid_mask].index:
            print(f"Solution: {solution.loc[idx, 'text']}")
            print(f"Submission: {submission.loc[idx, 'text']}")
            print(f"Solution Count: {sol_counts[idx]}")
            print(f"Submission Count: {sub_counts[idx]}")

        raise ParticipantVisibleError('At least one submitted string is not a valid permutation of the solution string.')

    # Calculate perplexity for the submitted strings
    sub_strings = [' '.join(s.split()) for s in submission['text'].tolist()]  # Normalize whitespace

    scorer = PerplexityCalculator(model_path=model_path, load_in_8bit=load_in_8bit)

    # Get perplexities for each submitted string
    perplexities = scorer.get_perplexity(sub_strings)

    if clear_mem:
        try:
            scorer.clear_gpu_memory()
        except Exception as e:
            print(f'GPU memory clearing failed: {e}')

    return float(np.mean(perplexities))

# Example usage with provided sentences (replace with actual DataFrame inputs)
model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
solution_df = pd.DataFrame({
    'id': [0],
    'text': ["Minimizing perplexity, a task quite grand. A neural network’s mission, across the land. A model, trained on texts, both vast and deep. To rearrange the words, while others sleep. It sorts and sifts, with algorithms bright. To find the clearest path, the words made right. A digital reindeer, with circuits aglow. Reducing confusion, a wondrous show. No longer puzzled, the reader’s mind at ease. As the model’s magic, effortlessly please. A gift of clarity, a present divine. A neural network’s work, a clever design."]
})

submission_df = pd.DataFrame({
    'id': [0],
    'text': ["A neural network’s mission, across the land. Minimizing perplexity, a task quite grand. It sorts and sifts, with algorithms bright. To find the clearest path, the words made right."]
})

# Calculate score (perplexity) for submission against solution
try:
    perplexity_score = score(solution_df, submission_df, 'id', clear_mem=True)
    print(f'Mean Perplexity Score: {perplexity_score}')
except ParticipantVisibleError as e:
    print(e)

In [ ]:
import gc
import os
from math import exp
from collections import Counter
from typing import List, Union
import numpy as np
import pandas as pd
import transformers
import torch

# Set environment variables for optimal performance
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ParticipantVisibleError(Exception):
    pass

class PerplexityCalculator:
    """Calculates perplexity of text using a pre-trained language model."""

    def __init__(self, model_path: str, load_in_8bit: bool = False):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

        # Load model with or without 8-bit quantization
        if load_in_8bit:
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path, quantization_config=quantization_config).to(DEVICE)
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(model_path).to(DEVICE)

        self.model.eval()

    def get_perplexity(self, input_texts: Union[str, List[str]], batch_size: int = 4) -> Union[float, List[float]]:
        """Calculates the perplexity of given texts."""
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []
        with torch.no_grad():
            for i in range(0, len(input_texts), batch_size):
                batch_texts = input_texts[i:i + batch_size]
                for text in batch_texts:
                    # Add special tokens for boundary conditions
                    text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
                    model_inputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False)

                    # Move inputs to the appropriate device
                    model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                    # Get model output and calculate loss
                    output = self.model(**model_inputs)
                    logits = output.logits

                    # Shift logits and labels for calculating loss
                    shift_logits = logits[..., :-1, :].contiguous()
                    shift_labels = model_inputs['input_ids'][..., 1:].contiguous()

                    loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
                    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

                    # Calculate average loss
                    sequence_loss = loss.sum() / len(loss)
                    loss_list.append(sequence_loss.cpu().item())

        # Calculate perplexities from losses
        perplexities = [exp(i) for i in loss_list]
        return perplexities[0] if single_input else perplexities

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        del self.model
        gc.collect()
        torch.cuda.empty_cache()

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str,
          model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
          load_in_8bit: bool = False, clear_mem: bool = False) -> float:
    """Calculates the mean perplexity of submitted text permutations compared to an original text."""

    # Normalize text by lowercasing and stripping whitespace
    solution['text'] = solution['text'].str.lower().str.strip()
    submission['text'] = submission['text'].str.lower().str.strip()

    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)

    invalid_mask = sol_counts != sub_counts

    if invalid_mask.any():
        print("Mismatch found:")
        for idx in invalid_mask[invalid_mask].index:
            print(f"Solution: {solution.loc[idx, 'text']}")
            print(f"Submission: {submission.loc[idx, 'text']}")
            print(f"Solution Count: {sol_counts[idx]}")
            print(f"Submission Count: {sub_counts[idx]}")

        raise ParticipantVisibleError('At least one submitted string is not a valid permutation of the solution string.')

    # Calculate perplexity for the submitted strings
    sub_strings = [' '.join(s.split()) for s in submission['text'].tolist()]  # Normalize whitespace

    scorer = PerplexityCalculator(model_path=model_path, load_in_8bit=load_in_8bit)

    # Get perplexities for each submitted string
    perplexities = scorer.get_perplexity(sub_strings)

    if clear_mem:
        try:
            scorer.clear_gpu_memory()
        except Exception as e:
            print(f'GPU memory clearing failed: {e}')

    return float(np.mean(perplexities))

# Example usage with provided sentences (replace with actual DataFrame inputs)
model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
solution_df = pd.DataFrame({
    'id': [0],
    'text': ["Minimizing perplexity, a task quite grand. A neural network’s mission, across the land. A model, trained on texts, both vast and deep. To rearrange the words, while others sleep. It sorts and sifts, with algorithms bright. To find the clearest path, the words made right. A digital reindeer, with circuits aglow. Reducing confusion, a wondrous show. No longer puzzled, the reader’s mind at ease. As the model’s magic, effortlessly please. A gift of clarity, a present divine. A neural network’s work, a clever design."]
})

submission_df = pd.DataFrame({
    'id': [0],
    'text': ["A neural network’s mission, across the land. Minimizing perplexity, a task quite grand. It sorts and sifts, with algorithms bright. To find the clearest path, the words made right."]
})

# Calculate score (perplexity) for submission against solution
try:
    perplexity_score = score(solution_df, submission_df, 'id', clear_mem=True)
    print(f'Mean Perplexity Score: {perplexity_score}')
except ParticipantVisibleError as e:
    print(e)